In [ ]:
%pip install requests pandas

In [ ]:
import os
import itertools
import logging
from datetime import datetime, timedelta, date

import requests
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")

In [ ]:
token_env = 'test'
AMADEUS_CLIENT_ID = 'yfn83bY1wvh9l45ffVklwppFrIyPTwfZ'
AMADEUS_CLIENT_SECRET = 'l9Jn2WCbwAmo97FS'

# Endpoints
if token_env == 'test':
    token_url  = 'https://test.api.amadeus.com/v1/security/oauth2/token'
    offers_url = 'https://test.api.amadeus.com/v2/shopping/flight-offers'
else:
    token_url  = 'https://api.amadeus.com/v1/security/oauth2/token'
    offers_url = 'https://api.amadeus.com/v2/shopping/flight-offers'

print(f"Verwende {token_env}-Endpoint für Token: {token_url}")

# Zeitraum
dt_start = date(2025, 5, 1)
dt_end   = datetime.utcnow().date()
dates    = [
    (dt_start + timedelta(days=i)).isoformat()
    for i in range((dt_end - dt_start).days + 1)
]

# IATA-Codes
iata_codes = [
    "IST", "BER", "LHR", "CDG", "FCO", "MAD", "KBP", "WAW",
    "OTP", "AMS", "ARN", "BRU", "ATH", "PRG", "LIS", "BUD",
    "BEG", "VIE", "ZRH", "SOF", "CPH", "HEL", "OSL", "DUB",
]

print(f"Abfragen für {len(dates)} Tage von {dt_start} bis {dt_end} für {len(iata_codes)} Städte.")

In [ ]:
def get_access_token(client_id: str, client_secret: str) -> str:
    data = {'grant_type': 'client_credentials'}
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    resp = requests.post(
        token_url,
        data={**data, 'client_id': client_id, 'client_secret': client_secret}
        if token_env=='test' else data,
        headers=headers if token_env=='test' else None,
        auth=None if token_env=='test' else (client_id, client_secret),
        timeout=10
    )
    resp.raise_for_status()
    token = resp.json().get('access_token')
    if not token:
        raise RuntimeError(f"Kein access_token: {resp.text}")
    return token

In [ ]:
def get_flight_prices(token: str, origin: str, destination: str, departure_date: str) -> dict:
    resp = requests.get(
        offers_url,
        headers={'Authorization': f'Bearer {token}'},
        params={
            'originLocationCode': origin,
            'destinationLocationCode': destination,
            'departureDate':        departure_date,
            'adults':               1,
            'currencyCode':         'EUR',
            'max':                  250
        },
        timeout=10
    )
    resp.raise_for_status()
    return resp.json()

In [ ]:
def parse_response(data: dict) -> list:
    recs = []
    for offer in data.get('data', []):
        price = offer.get('price', {})
        itins = offer.get('itineraries', [])
        if not itins or not itins[0].get('segments'):
            continue
        segs = itins[0]['segments']
        recs.append({
            'FetchedAt':     datetime.utcnow().isoformat(),
            'Origin':        segs[0]['departure']['iataCode'],
            'Destination':   segs[-1]['arrival']['iataCode'],
            'DepartureDate': segs[0]['departure']['at'],
            'MinPrice':      price.get('total'),
            'Currency':      price.get('currency'),
            'Segments':      len(segs)
        })
    return recs


In [ ]:
token      = get_access_token(AMADEUS_CLIENT_ID, AMADEUS_CLIENT_SECRET)
city_data  = {o: [] for o in iata_codes}
MAX_RECORDS = 5
SKIP_CODES  = {400, 401, 403, 429}

for origin, destination in itertools.permutations(iata_codes, 2):
    # schon genug gesammelt?
    if len(city_data[origin]) >= MAX_RECORDS:
        continue

    for d in dates:
        # abbruch, wenn Limit erreicht
        if len(city_data[origin]) >= MAX_RECORDS:
            break

        try:
            data = get_flight_prices(token, origin, destination, d)
            recs = parse_response(data)
            if recs:
                city_data[origin].extend(recs)
        except requests.HTTPError as e:
            code = e.response.status_code
            # Bad Request, Too Many Requests etc. still überspringen
            if code not in SKIP_CODES:
                logging.warning(f"Fehler {origin}->{destination} am {d}: {e}")
        except Exception as e:
            logging.warning(f"Unerwarteter Fehler {origin}->{destination} am {d}: {e}")

    logging.info(f"{origin}: {len(city_data[origin])} Datensätze gesammelt")

In [53]:
os.makedirs('PricesCSV', exist_ok=True)
for origin, recs in city_data.items():
    if not recs:
        print(f"Keine Daten für {origin}")
        continue
    df = pd.DataFrame(recs)
    path = os.path.join('PricesCSV', f"amadeus_prices_{origin}.csv")
    df.to_csv(path, index=False)
    print(f"Gespeichert: {path} ({len(df)} Einträge)")


Keine Daten für AMS
Keine Daten für ARN
Keine Daten für BRU
Keine Daten für ATH
Keine Daten für PRG
Keine Daten für LIS
Keine Daten für BUD
Keine Daten für BEG
Keine Daten für VIE
Keine Daten für ZRH
Keine Daten für SOF
Keine Daten für CPH
Keine Daten für HEL
Keine Daten für OSL
Keine Daten für DUB


In [52]:
import glob
files = sorted(glob.glob('PricesCSV/amadeus_prices_*.csv'))
if files:
    display(pd.read_csv(files[0]).head())
else:
    print("Keine CSVs in PricesCSV gefunden.")

,FetchedAt,Origin,Destination,DepartureDate,MinPrice,Currency,NumberOfSegments
0,2025-06-19T19:13:31.487289,BER,IST,2025-06-19T21:15:00,223.07,EUR,2
1,2025-06-19T19:13:31.487307,BER,IST,2025-06-19T21:15:00,223.07,EUR,2
2,2025-06-19T19:13:31.487312,BER,IST,2025-06-19T21:15:00,500.19,EUR,2
3,2025-06-19T19:14:04.712096,BER,LHR,2025-06-19T21:20:00,255.82,EUR,2
4,2025-06-19T19:14:04.712113,BER,LHR,2025-06-19T21:20:00,255.82,EUR,2
